In [1]:
import re
import requests
import json
from collections import defaultdict


def form_url(ISBN=None):
    """Formulate an url that will be sent to openlibrary.org/api'.

    Parameters
    ----------
    ISBN : string
        a string of book's ISBN.

    Returns
    -------
    string
        An url that will be sent to openlibrary/org/api.

    """
    url = 'https://openlibrary.org/api/books?bibkeys=ISBN:' + \
        ISBN + '&jscmd=data&format=json'

    return url

def form_url_title(title='here is where we meet'):
    """Formulate an url that will be sent to openlibrary.org/api'.

    Parameters
    ----------
    ISBN : string
        a string of book's ISBN.

    Returns
    -------
    string
        An url that will be sent to openlibrary/org/api.

    """
    title = "+".join( title.split() )
    url = 'https://isbnsearch.org/search?s=' + \
        title

    return url


def get_ISBN_from_title(title):
    h = {'Authorization': '43360_fd60754106422e4ff2600025312a1118'}
    title = title.title()
    #title = "%20".join( title.split() )
    resp = requests.get("https://api2.isbndb.com/books/{" \
             +title +"}", headers=h)
    # print('Acquired following info about the book {}'.format(resp.))
    # print(resp.json())
    results = resp.json()['books']
    
    return results

def get_author_name(results):
    authors = []
    for x in results:
        try:
            if 'authors' in x.keys():
                authors.append(x['authors'])
        except Exception:
            return "Jane Doe"
                    
    return authors

def mode_authors(list_of_authors):
    author_count = defaultdict(int)
    for auth in list_of_authors:
        for x in auth: 
            author_count[x] += 1 
        
    return max(author_count, key=lambda key: author_count[key])
    
def extract_IBSN_from_api_return(results, author):
    # note that this gets the last ISBN in the list--there maybe multiple copies
    # of the book--hopefully all additions have the same dewey decimal number
    ISBN = None
    right = []
    genre = None

    for x in results:
        #print(x.keys)
        try:
            if 'authors' in x.keys():
                if author in x['authors']:
                    # print(x)
                    right.append(x['isbn13'])
                    right.append(x['isbn'])
                    
                    if 'subjects' in x.keys():
                        genre = x['subjects']

        
        except Exception:
            pass

    return right, genre


def get_response(url):
    response = requests.get(url)
    # print('Acquired following info about the book {}'.format(response.text))
    return response.text


def extract_ddc(text):
    try:
        dewey_pattern = re.compile(r'(\\d{3}/.\\d+?\/?d)')
        ddc = dewey_pattern.findall(text)[0]
        # print('Dewey decimal code of the book is {}'.format(ddc))
    except:
        if 'fiction' in text:
            return 813.
        else:
            print("dewey_decimal_not_found")
            ddc = None
    
    return ddc

def check_genre_ddc(genre):
    for x in genre:
        # print(x)
        if x.lower() == 'fiction':
            return 813.6
        if x.lower() == 'nonfiction':
            return 810
        if x.lower() == 'journalism':
            return '070' 
        
    return 000.0


def get_ddc_api(ISBN=None, title=None, author_name=None):

    ddc = None
   
    if ISBN is not None:
        url = form_url(ISBN)
        response = get_response(url)
        ddc = extract_ddc(response)
        return ddc
    
    else:
        response = get_ISBN_from_title(title)
        if author_name is None:
            author_name = mode_authors(get_author_name(response))
            print(author_name)
            
        ISBN, genre = extract_IBSN_from_api_return(response, author_name)
        print(ISBN, genre)
        #print(ISBN)
        for x in ISBN:
            if not ddc:
                url = form_url(x)
                response = get_response(url)
                ddc = extract_ddc(response)
                if ddc is not None:
                    return ddc
                else:
                    pass
        if ddc is None and genre is not None:
            ddc = check_genre_ddc(genre)
            return ddc
            
    return ddc


In [2]:
get_ddc_api(title="The Power of Six")

Pittacus Lore
['9780718157951', '0718157958', '9780062111630', '0062111639', '9780606262897', '060626289X', '9780141047850', '0141047852', '9780141338668', '0141338660', '9780061974557', '0061974552', '9780141330877', '0141330872'] ['Extraterrestrial Beings', 'Convents', 'Fugitives From Justice', 'Friendship', 'Science Fiction']
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found


813.0

In [3]:
get_ddc_api(title="Diary of a Wimpy Kid RODRICK")

Jeff Kinney
['9780810982932', '0810982935', '9780810988941', '0810988941', '9781436196321', '1436196329', '9780670074938', '0670074934', '9780810994737', '0810994739', '9781419714115', '1419714112', '9780810987999', '0810987996', '9780141336343', '014133634x', '9781419707667', '1419707663', '9781439582640', '1439582645', '9789868484740', '986848474X', '9780606236638', '0606236635', '9781933032627', '1933032626', '9788993055030', '8993055033', '9780143304555', '0143304550', '9780810996168', '0810996162', '9781615579181', '1615579184', '9780141324920', '0141324929'] ['Fiction - Games & Activities', 'Fiction - Schools & Friendship', 'Fiction - Comic Books & Graphic Novels']
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found


813.0

In [4]:
extract_IBSN_from_api_return(get_ISBN_from_title("Silence"), 'Shusaku Endo')

(['9780720612868', '0720612861', '9780870115356', '0870115359'], ['Fiction'])

In [5]:
get_ddc_api(title="Silence", author_name="Shusaku Endo")

['9780720612868', '0720612861', '9780870115356', '0870115359'] ['Fiction']
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found


813.6

In [6]:
get_ddc_api(title="A Personal History", author_name="Katharine Graham")

['9781842126011', '1842126016'] ['Language Arts &amp; Disciplines', 'Journalism']
dewey_decimal_not_found
dewey_decimal_not_found


'070'

In [7]:
get_ddc_api(title="Python Machine Learning", author_name="Sebastian Raschka")

['9781787126022', '1787126021', '9781783555130', '1783555130'] None
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found
dewey_decimal_not_found


In [8]:
get_ddc_api(title="A Tale of Two Cities", author_name="Charles Dickens")

['9780816130757', '0816130752', '9781576467374', '1576467376', '9781615701551', '1615701559', '9781428108349', '1428108343', '9781609420598', '1609420594', '9780448060231', '044806023X', '9781279853146', '127985314x', '9781578152452', '1578152453', '9780710517395', '0710517394', '9780451519597', '0451519590', '9781593351830', '1593351836', '9781420922363', '142092236X', '9780681995628', '0681995629', '9780886460433', '0886460433', '9781426451232', '1426451237'] ["Children's Fiction", 'Classics']


813.0